In [1]:
#Importing Dependencies
import pandas as pd
import requests
import datetime 
import json

#Start Dates
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 12, 1)
delta = datetime.timedelta(days=25)

start_list =[]

while start_date <= end_date:
    start_list.append(start_date)
    start_date += delta

#End Dates
start_date = datetime.date(2021, 1, 26)
end_date = datetime.date(2021, 12, 1)
delta = datetime.timedelta(days=25)

end_list =[]

while start_date <= end_date:
    end_list.append(start_date)
    start_date += delta

#Converting lists to strings 
end_str_list = []
for i in range(0,len(end_list)):
    end_str_list.append(end_list[i].strftime('%Y%m%d'))
    
start_str_list =[]
for i in range(0,len(start_list)):
    start_str_list.append(start_list[i].strftime('%Y%m%d'))

# Code for 2021 data
start_list = pd.date_range(start='1/1/2021', end ='1/1/2022', freq='MS')

start_str_list = []
end_str_list = []
for date in start_list:
    start_str_list.append(date.strftime('%Y%m%d'))
    end_str_list.append(date.strftime('%Y%m%d'))

start_str_list = start_str_list[0:12]
end_str_list = end_str_list[1:13]

In [2]:
# Code for 60 days' retro data
today = pd.to_datetime("today") + datetime.timedelta(days=1)  

start_str_list = []
end_str_list = []

retro_start = 60
retro_end = 30

# For start dates
for x in range(0,2):
    if retro_start >= 0:
        start = today - datetime.timedelta(days=retro_start)
        start_str_list.append(start.strftime('%Y%m%d'))
        retro_start -=30
        
    else:
        pass

# For end dates
for y in range(0,1):
    if retro_end > 0:
        end = today - datetime.timedelta(days=retro_end)
        end_str_list.append(end.strftime('%Y%m%d'))        
        retro_end -=30
        
    else:
        pass

end_str_list.append(today.strftime('%Y%m%d'))

In [3]:
#List of endpoints for API
complete_endpoints = []
for i in range(0,len(start_str_list)):
    complete_endpoints.append(f"{start_str_list[i]}:{end_str_list[i]}")

In [4]:
#Base URL setup
base_url = "https://www.ncdc.noaa.gov/swdiws/json/nx3tvs/"

In [5]:
#List of all URLs
all_urls = []
for dates in complete_endpoints:
    complete_url = base_url + dates
    all_urls.append(complete_url)
#print(len(all_urls))

In [6]:
#List/Variables setup
cell_type =[]
shape =[]
max_shear = []
wsr_id =[]
mxdv = []
cell_id =[]
ztime = []
azimuth = []
range_ = []
lat = []
lon = []

#url_counter = 1

# Start of Log
print("-----------------------------")
print("Beginning Data Retrieval - estimated time is <5min")
print("-----------------------------")

#----BEGIN Loop----
# Looping through urls and appending data to lists
for url in all_urls:
    try:
        response = requests.get(url).json()
        for i in range(len(response["result"])):
            cell_type.append(response["result"][i]["CELL_TYPE"])
            shape.append(response["result"][i]["SHAPE"])
            max_shear.append(response["result"][i]["MAX_SHEAR"])
            wsr_id.append(response["result"][i]["WSR_ID"])
            mxdv.append(response["result"][i]["MXDV"])
            cell_id.append(response["result"][i]["CELL_ID"])
            ztime.append(response["result"][i]["ZTIME"])
            azimuth.append(response["result"][i]["AZIMUTH"])
            range_.append(response["result"][i]["RANGE"])
            lon.append(response["result"][i]["SHAPE"].split()[1:][0].split('(')[1])
            lat.append(response["result"][i]["SHAPE"].split()[1:][1].split(')')[0])
            #print(f"Processing Record {i} of URL {url_counter}")
        #url_counter += 1
    
    except(KeyError, IndexError):
        print(f"URL {url} not found. Skipping...")

#----END Loop-----

# End of Log
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------") 

-----------------------------
Beginning Data Retrieval - estimated time is <5min
-----------------------------
-----------------------------
Data Retrieval Complete
-----------------------------


In [7]:
#Creating dataframe of all records
dict = {"Cell_Type": cell_type,
        "Shape": shape,
        "Max_Shear": max_shear,
        "WSR_ID": wsr_id,
        "MXDV": mxdv,
        "Cell_ID": cell_id,
        "zTime": ztime,
        "Azimuth": azimuth,
        "Range": range_,
        "Lat": lat,
        "Lon": lon
       }

df = pd.DataFrame(dict)
#df

In [8]:
#Creating additional columns
df['Date']=pd.to_datetime(df['zTime']).dt.date
df['Time']=pd.to_datetime(df['zTime']).dt.time
df['Year']=pd.to_datetime(df['zTime']).dt.year
df['Month']=pd.to_datetime(df['zTime']).dt.month

In [9]:
#Primary Key to id tornados
df['PKID']=df['Cell_ID']+df['zTime'].astype(str)
#df.head()

In [10]:
df2 = df.sort_values(['Date','Time'], ascending = [True,True])
df2

,Cell_Type,Shape,Max_Shear,WSR_ID,MXDV,Cell_ID,zTime,Azimuth,Range,Lat,Lon,Date,Time,Year,Month,PKID
0,TVS,POINT (-97.3094240935169 27.9825363727376),62,KCRP,60,C4,2022-04-26T00:21:50Z,42,16,27.9825363727376,-97.3094240935169,2022-04-26,00:21:50,2022,4,C42022-04-26T00:21:50Z
1,TVS,POINT (-93.3900788628959 30.4187223805647),17,KPOE,53,G2,2022-04-26T00:30:54Z,206,49,30.4187223805647,-93.3900788628959,2022-04-26,00:30:54,2022,4,G22022-04-26T00:30:54Z
2,TVS,POINT (-97.5768374099097 27.6638333872168),101,KCRP,51,G1,2022-04-26T01:31:45Z,206,8,27.6638333872168,-97.5768374099097,2022-04-26,01:31:45,2022,4,G12022-04-26T01:31:45Z
3,TVS,POINT (-97.2161067129873 25.9286832990756),78,KBRO,55,S3,2022-04-26T09:56:32Z,86,11,25.9286832990756,-97.2161067129873,2022-04-26,09:56:32,2022,4,S32022-04-26T09:56:32Z
4,TVS,POINT (-79.6316207684386 35.8085966958722),32,KRDU,95,H0,2022-04-26T18:19:06Z,256,47,35.8085966958722,-79.6316207684386,2022-04-26,18:19:06,2022,4,H02022-04-26T18:19:06Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31369,TVS,POINT (-78.2689913686982 39.4541144629246),33,KIAD,86,C2,2022-06-22T23:48:49Z,303,41,39.4541144629246,-78.2689913686982,2022-06-22,23:48:49,2022,6,C22022-06-22T23:48:49Z
31370,TVS,POINT (-78.2886395928353 39.574714768867),32,KIAD,92,R0,2022-06-22T23:48:49Z,310,46,39.574714768867,-78.2886395928353,2022-06-22,23:48:49,2022,6,R02022-06-22T23:48:49Z
31371,TVS,POINT (-78.0694612566902 39.5810564397342),28,KIAD,70,E4,2022-06-22T23:48:49Z,320,39,39.5810564397342,-78.0694612566902,2022-06-22,23:48:49,2022,6,E42022-06-22T23:48:49Z
31372,TVS,POINT (-78.242657983444 39.4549034063783),34,KIAD,86,C2,2022-06-22T23:51:39Z,304,40,39.4549034063783,-78.242657983444,2022-06-22,23:51:39,2022,6,C22022-06-22T23:51:39Z


#Function to build geojson
def df_to_geojson(df, properties, lat='Lat', lon='Lon'):
    #Create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    #Loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        #Create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        #Fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        #For each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        #Add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

# Setting columns and running function
cols = ["Cell_Type", "Shape", "Max_Shear", 
        "WSR_ID", "MXDV", "Cell_ID", 
        "zTime", "Azimuth", "Range", "Lat", "Lon",
        "Date","Time","Year","Month","PKID"]
geojson = df_to_geojson(df, cols)

#Saving output into js file
output_filename = 'static/data/tornadoes.geojson'
with open(output_filename, 'w') as output_file:
    json.dump(geojson, output_file, indent=2, default=str)

In [11]:
df.to_json('static/data/tornadoes.json', orient = 'records', compression = 'infer', index = 'true')